## 確保應用程式的穩定性

In [6]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,3,25,2024-07-02 15:33:21
1,大安區,21,8,13,2024-07-02 14:40:30
2,大安區,16,4,12,2024-07-02 14:01:23
3,大安區,11,2,9,2024-07-02 14:23:08
4,大安區,16,3,13,2024-07-02 15:22:20
...,...,...,...,...,...
1425,臺大公館校區,30,18,0,2024-07-02 14:38:14
1426,臺大公館校區,20,20,0,2024-07-02 14:52:14
1427,臺大公館校區,24,13,3,2024-07-02 15:32:16
1428,臺大公館校區,40,31,0,2024-07-02 15:25:16


In [7]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

#class 的定義建議寫在程式上面
class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday') #python 裡面有datetime這個型別

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value

class Youbike(RootModel): #根是list 所以要用Rootmodel
    root:list[Site]

#確認連線的穩定性
try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status() #如果連線有問題就會出現錯誤
except Exception as e :
    print(e)
else:
    data=Youbike.model_validate_json(youbike_url.text) #驗證資料
    all_sites = data.model_dump() #轉成python 的資料結構

    df = pd.DataFrame(all_sites)
    df




In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1430 entries, 0 to 1429
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   行政區域    1430 non-null   object        
 1   總數量     1430 non-null   int64         
 2   可借數量    1430 non-null   int64         
 3   可還數量    1430 non-null   int64         
 4   時間      1430 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 56.0+ KB


In [9]:
df1 = df.set_index(keys=["行政區域"])
df1.groupby(level='行政區域')[['總數量','可借數量','可還數量']].sum()

#用指定的方式,進行計算。

,總數量,可借數量,可還數量
行政區域,,,
中山區,3819,1718,2058
中正區,3209,1624,1517
信義區,3331,1415,1802
內湖區,3499,1572,1874
北投區,2472,858,1564
南港區,2317,885,1424
士林區,2962,1314,1596
大同區,1630,554,1066
大安區,4987,2253,2676
